## Import Modules

In [1]:
import pdal
import sys
import os
import json
sys.path.append("../../../PythonScripts")
from glob import glob
import open3d as o3d
import os
import numpy as np
import pandas as pd
from multiprocessing import Pool
from pipeline_functions import downscale_las, get_scales, downscale_ply, getFeatures,getFeaturesParallel,Cylinder,get_scales_cyl
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from time import time
# from scipy.spatial import cKDTree
from sklearn.neighbors import KDTree, BallTree
# from scipy.spatial import cKDTree
import pickle
from datetime import datetime, timezone
from itertools import repeat # Used for starmap function
import gc # Used to free up memory to prevent kernel restarting

sns.set()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Get number of processors to run loop

In [2]:
processors = os.cpu_count() - 12

## Get File to run

In [3]:
# ROOT = """/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117.las"""
ROOT = """/home/sspiegel/CapstoneData/Paris/training_10_classes/Lille1_1.ply"""
# ROOT = """/home/sspiegel/CapstoneData/PA/225019325.las"""

## Load in base point cloud

In [4]:
pipeline_json = [
    {
        "type": "readers.ply",
        "filename": ROOT
    }
        
]


pipeline = pdal.Pipeline(json.dumps(pipeline_json))

# Execute the pipeline
# This will process the data according to the stages defined in the pipeline
pipeline.execute()

xy = pipeline.arrays[0]

mta = pipeline.metadata



In [5]:
xyzT = np.array((xy["X"], xy["Y"], xy["Z"])).T

In [6]:
xyz = xyzT - np.min(xyzT, axis = 0)

## Get Classification

In [7]:
cls = xy["Classification"].astype(int)

In [8]:
scls = get_scales_cyl(S = 6) # Use base paprameters from function

In [9]:
scls

[(0.1414, 0.0283),
 (0.2828, 0.0566),
 (0.5657, 0.1131),
 (1.1314, 0.2263),
 (2.2627, 0.4525),
 (4.5255, 0.9051)]

## Instantiate point cloud

In [10]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

In [11]:
pc_dict = []
for s in scls:
    
    pc = pcd.voxel_down_sample(s[1])
    # pc_array = np.asarray(pc.points)
    pc_dict.append({
      "r" : s[0],
     "grid_size" : s[1],
     "point_cloud" : pc
     # "point_cloud_array" : pc_carray
    })


## Get scaling factors

### Downscale point clouds: Methodology

* Downscale point clouds based on multiscaling factor
* The scaling factor chosen from [Thomas Hugues et. al](https://ieeexplore.ieee.org/document/8490990/)
* Parameters
  * initial radius ($r_0$): 0.1
  * Number of scales ($S$): 6
  * Base exponent for expanding radius ($\gamma$): 2
  * scaling factor of grid ($\rho$): 5
  * radius of sphere for scale $s$: $r_s = r_0 * \gamma^{s}$
  * downsampled voxel size: $\frac{r_s}{\rho}$

* Use multiprocessing to speed up process

### Sample point cloud (first radius)

In [12]:
test = pc_dict[2]

test_pc = test["point_cloud"]

cyl = Cylinder(R = test['r'])

tree = KDTree(test_pc.points)
# tree = cKDTree(CylPoints)

In [13]:
print(test)

{'r': 0.5657, 'grid_size': 0.1131, 'point_cloud': PointCloud with 3767523 points.}


In [14]:
test_pc_arr = np.asarray(test_pc.points)

## Get Distance matrix

In [15]:
# cylPCDPoints = np.asarray(pcd.points)[:,:2]

In [16]:
distMat = tree.query_radius(pcd.points, r=test["r"])



# distMat = tree.query_ball_point(pcd.points, r=test["r"], workers = 1)


In [17]:
# distMat

In [18]:
# args1 = zip(repeat(pcOut),repeat(test_pc_arr), distMat)
# with Pool(processes=processors) as pool1:
#     result1 = pool1.starmap(cyl.computePoints, args1)
# distMatFix = np.array(result1)

In [19]:
pcOut = np.asarray(pcd.points)


In [20]:
del pc_dict, xyzT, xyz, xy, pcd

gc.collect()

32

In [21]:
# cyl.computePointsParallel(pcOut[0],test_pc_arr,distMat[0])

In [22]:
args1 = zip(pcOut,repeat(test_pc_arr), distMat)
with Pool(processes=processors) as pool1:
    result1 = pool1.starmap(cyl.computePointsParallel, args1)
distMatFix = np.asarray(result1, dtype = "object")

In [23]:
gc.collect()

0

In [24]:
# distMat.dtype

In [25]:
# distMatFix = np.asarray(result1, dtype = "object")

In [26]:
# distMatFix[0]
# # 

In [27]:
# distMat[0]

In [28]:
# pcOut[0]

In [29]:
# ids = np.random.randint(0, pcOut.shape[0], 1)

In [30]:
# ids

In [31]:
# pcCenter = o3d.geometry.PointCloud()
# pcDown = o3d.geometry.PointCloud()
# pcCyl = o3d.geometry.PointCloud()

# pcCenter.points = o3d.utility.Vector3dVector(pcOut[ids[0]].reshape(1,-1))
# pcDown.points = o3d.utility.Vector3dVector(test_pc_arr[distMat[ids[0]]])
# pcCyl.points = o3d.utility.Vector3dVector(test_pc_arr[distMatFix[ids[0]]])

# pcCenter.paint_uniform_color([1,0,0])
# pcDown.paint_uniform_color([0,1,0])
# pcCyl.paint_uniform_color([0,0,1])

# o3d.visualization.draw_geometries([pcCenter, pcDown, pcCyl])


In [32]:
del distMat

In [33]:
gc.collect()

0

## Split and apply features algorithmKDTree

In [34]:
# DIV = 100000
# groups = di stMatFix.shape[0] // DIV

In [35]:
s = time()

bigList = []

# for g in range(groups + 1):
#     print("""Processing batch %d...""" % (g+1))
#     practiceList = distMatFix[g*DIV:(g+1)*DIV]

args = zip(repeat(test_pc_arr), distMatFix)
with Pool(processes=processors) as pool:
    bigList = pool.starmap(getFeaturesParallel, args)
    # result = np.array(bigList)
    # bigList.append(result)
    # print(practiceList.shape)
    # break

bigList = np.array(bigList)
e = time()

print("""\n\n\n\nDone!!!  Total time: %.4f seconds""" % (e - s))





Done!!!  Total time: 813.1570 seconds


In [36]:
bigList[:3,:]

array([[6.27462640e-02, 1.37570681e-02, 2.23327202e-01, 5.81706613e-01,
        3.32571940e-01, 8.57214231e-02, 5.69950676e-02, 5.18144368e-01,
        9.31814346e-01, 8.79105859e-03, 4.14613088e-01, 6.00000000e+01],
       [6.63583590e-02, 1.51249601e-02, 2.35371685e-01, 4.53179299e-01,
        4.50807243e-01, 9.60134330e-02, 5.84437795e-02, 5.17186732e-01,
        9.30283841e-01, 1.07204778e-02, 4.44954236e-01, 6.70000000e+01],
       [6.56654374e-02, 1.71882867e-02, 2.38031459e-01, 4.67028923e-01,
        3.68350918e-01, 1.64620133e-01, 9.69727794e-02, 5.44822772e-01,
        9.13139342e-01, 1.19000332e-02, 4.74704107e-01, 6.70000000e+01]])

## Save as a pickle file

In [37]:
del distMatFix, tree

gc.collect()

0

In [38]:
# pickleOut = f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/pickleFiles/{datetime.now(timezone.utc).strftime("%Y_%m_%dT%H_%M")}_L001_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}.pkl"""

In [39]:
getTime = datetime.now(timezone.utc).strftime("%Y_%m_%dT%H_%M")

featureOut = f"""/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/cylinder/training/{getTime}_Lille1_1_cylinder_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_features.npz"""
# pickleOutPC = f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/pickleFiles/{getTime}_L001_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_pointcloud.pkl"""

# outDict = { "features" : bigList, "classification" : cls}
# # outDict = {"PointCloud" : pcOut}


# with open(pickleOut, 'wb') as f:
#     pickle.dump(outDict, f)

np.savez(featureOut,array1 = pcOut, array2 = bigList, array3 = cls)

## Save stats

In [40]:
sts = f"""/home/sspiegel/CapstoneData/Paris/training_10_classes/stats/{datetime.now(timezone.utc).strftime("%Y_%m_%d")}_cylinder_r_{str(test["r"]).split('.')[0]}_{str(test["r"]).split('.')[1]}_grid_{str(test["grid_size"]).split('.')[0]}_{str(test["grid_size"]).split('.')[1]}_features.json"""


In [41]:
jsStats = {
    "filename" : ROOT,
    "radius" : test["r"],
    "grid_size" : test["grid_size"],
    "total_time" : "%.4f seconds" % (e - s)
}

In [42]:
with open(sts, "w") as jsOut:
    jsOut.write(json.dumps(jsStats))
